In [7]:
# IMPORTS

import numpy as np 
import pandas as pd 
import random as rd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [8]:
# PRE-PROCESSING

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

table_to_drop = ['Name', 'Destination', 'Spa', 'VRDeck']
def drop_tables(tables, train_df):
    train_df = train_df.drop(columns=tables)
    return train_df

train_df = drop_tables(table_to_drop, train_df)
train_df = train_df.dropna(how = 'any')
test_df = drop_tables(table_to_drop, test_df)
test_df = test_df.dropna(how = 'any')

tables_to_encode = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'VIP', 'Transported']
def encode_tables(tables, df):
    copy = train_df
    for table in tables:
        copy[table] = copy[table].astype('category')
        copy[table] = copy[table].cat.codes
        copy[table] = copy[table].astype('float64')
    return copy

train_df = encode_tables(tables_to_encode, train_df)
test_df = encode_tables(tables_to_encode, test_df)

labels = train_df['Transported']
train_df = train_df.drop('Transported', axis=1)

In [9]:
# TRAINING

y_train = labels.to_numpy()
x_train = train_df.to_numpy()
x_test = test_df.to_numpy()
y_test = test_df.to_numpy()


rf_classifier = RandomForestClassifier()
rf_classifier.fit(x_train, y_train)

print(rf_classifier.score(x_train, y_train))

1.0
